In [1]:
import sys
!{sys.executable} -m pip install sparqlwrapper

In [2]:
import requests
import yaml
from string import Template
from SPARQLWrapper import SPARQLWrapper, JSON, TURTLE
from urllib.parse import quote
from os.path import join

In [3]:
configFile = '../config/config.yml'

In [4]:
with open(configFile, 'r') as f:
    config = yaml.safe_load(f)

In [5]:
def sparqlResultToDict(results):
    rows = []
    for result in results["results"]["bindings"]:
        row = {}
        for key in list(result.keys()):
            row[key] = result[key]["value"]
        rows.append(row)
    return rows

In [6]:
sparql = SPARQLWrapper(config['endpoint'])
sparql.setReturnFormat(JSON)

In [7]:
sparql.setQuery(config['graphsQuery'])
results = sparql.query().convert()

In [8]:
graphs = [d['g'] for d in sparqlResultToDict(results)] + config['additionalGraphs']

In [9]:
constructTemplate = Template("""
CONSTRUCT {
    ?s ?p ?o .
} WHERE {
  GRAPH <$graph> {
      ?s ?p ?o
  }
}
""")


In [10]:
headers = {
    "Accept": "application/x-trig"
}

queryTemplate = Template("""CONSTRUCT {
        ?s ?p ?o
        } FROM <$graph> WHERE {
            ?s ?p ?o .
        }""")

for graph in graphs:
    data = {
        "query": queryTemplate.substitute(graph=graph)
    }
    try:
        response = requests.get(config['endpoint'], headers=headers, params=data)
    except:
        print("Could not fetch data")
    if response:
        output = response.content.decode().replace("{", "<" + graph + "> {")
        filename = quote(graph, safe='') + '.trig'
        with open(join('..', config['output'], filename), 'w') as f:
            f.write(output)